# Lab 2: Create a Simple AI Agent

In this lab, we'll introduce you to AI agents by creating a simple agent that will create a bar graph based on data that we give to it. 

#### Step 1: Load packages

In [7]:
import os
from typing import Any
from pathlib import Path
from dotenv import load_dotenv
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.projects.models import CodeInterpreterTool

load_dotenv() # Load environment variables from .env file

True

#### Step 2: Connect to your Azure AI Foundry project

In [8]:
# Connecting to our Azure AI Foundry project, which will allow us to use the deployed gpt-4o model
project_connection_string = os.getenv("AIPROJECT_CONNECTION_STRING")
model = os.getenv("CHAT_MODEL")

project_client = AIProjectClient.from_connection_string(
    conn_str=project_connection_string, credential=DefaultAzureCredential()
)

#### Step 3: Create the simple AI Agent

In [9]:
with project_client:
    # Create an instance of the CodeInterpreterTool, which is responsible for generating the bar chart
    code_interpreter = CodeInterpreterTool()

    # The CodeInterpreterTool needs to be included in creation of the agent so that it can be used
    agent = project_client.agents.create_agent(
        model=model,
        name="my-agent-barchart",
        instructions="You are a helpful agent.",
        tools=code_interpreter.definitions,
        tool_resources=code_interpreter.resources,
    )
    print(f"Created agent, agent ID: {agent.id}")

    # Create a thread which is a conversation session between an agent and a user.
    thread = project_client.agents.create_thread()
    print(f"Created thread, thread ID: {thread.id}")

    # Create a prompt which contains the data + details for how the agent should generate the bar chart
    prompt = """Could you please create a bar chart for the using the following data and \
        provide the file to me? Name the file as finance-comparision.png. \
        Here is the data: \
  | Segment          | Country                  | Product   | Discount Band   |   Units Sold |   Manufacturing Price |   Sale Price |      Gross Sales |   Discounts |            Sales |     COGS |    Profit | Date                |   Month Number | Month Name   |   Year |
|:-----------------|:-------------------------|:----------|:----------------|-------------:|----------------------:|-------------:|-----------------:|------------:|-----------------:|---------:|----------:|:--------------------|---------------:|:-------------|-------:|
| Government       | Canada                   | Carretera | nan             |       1618.5 |                     3 |           20 |  32370           |       0     |  32370           |  16185   |  16185    | 2014-01-01 00:00:00 |              1 | January      |   2014 |
| Government       | Germany                  | Carretera | nan             |       1321   |                     3 |           20 |  26420           |       0     |  26420           |  13210   |  13210    | 2014-01-01 00:00:00 |              1 | January      |   2014 |
| Midmarket        | France                   | Carretera | nan             |       2178   |                     3 |           15 |  32670           |       0     |  32670           |  21780   |  10890    | 2014-06-01 00:00:00 |              6 | June         |   2014 |
| Midmarket        | Germany                  | Carretera | nan             |        888   |                     3 |           15 |  13320           |       0     |  13320           |   8880   |   4440    | 2014-06-01 00:00:00 |              6 | June         |   2014 |
| Midmarket        | Mexico                   | Carretera | nan             |       2470   |                     3 |           15 |  37050           |       0     |  37050           |  24700   |  12350    | 2014-06-01 00:00:00 |              6 | June         |   2014 |
| Government       | Germany                  | Carretera | nan             |       1513   |                     3 |          350 | 529550           |       0     | 529550           | 393380   | 136170    | 2014-12-01 00:00:00 |             12 | December     |   2014 |
    """
    
    # Create a message, with the prompt being the message content that is sent to the model
    message = project_client.agents.create_message(
        thread_id=thread.id,
        role="user",
        content=prompt,
    )
    print(f"Created message, message ID: {message.id}")

    # Run the agent to process tne message in the thread
    run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)
    print(f"Run finished with status: {run.status}")

    if run.status == "failed":
        # Check if you got "Rate limit is exceeded.", then you want to increase the token limit
        print(f"Run failed: {run.last_error}")

    # Get all messages from the thread
    messages = project_client.agents.list_messages(thread_id=thread.id)
    print(f"Messages: {messages}")

    #create output directory if it doesn't exist
    output_dir = Path.cwd() / "output"
    output_dir.mkdir(parents=True, exist_ok=True)

    # Generate an image file for the bar chart
    for file_path_annotation in messages.file_path_annotations:
        file_name = Path(file_path_annotation.text).name
        project_client.agents.save_file(file_id=file_path_annotation.file_path.file_id, file_name=file_name, target_dir=output_dir)
        print(f"Saved image file to: {Path.cwd() / "output" / file_name}")

    # Delete the agent once done
    project_client.agents.delete_agent(agent.id)
    print("Deleted agent")

Created agent, agent ID: asst_AYDTQO8xe8bq0Aq3PFcOvsEM
Created thread, thread ID: thread_1fmV8oW8ZT08fRIjwiQyE7jz
Created message, message ID: msg_FOLH3FXrNkiaFiYJSmqr4RgO
Run finished with status: RunStatus.COMPLETED
Messages: {'object': 'list', 'data': [{'id': 'msg_JR5zyXUMpk48dv2Ru31jZBB4', 'object': 'thread.message', 'created_at': 1742314829, 'assistant_id': 'asst_AYDTQO8xe8bq0Aq3PFcOvsEM', 'thread_id': 'thread_1fmV8oW8ZT08fRIjwiQyE7jz', 'run_id': 'run_WZ15npiCiyoNxxyIUkSqHedA', 'role': 'assistant', 'content': [{'type': 'image_file', 'image_file': {'file_id': 'assistant-BzE1DMYqWDAsY44AFEMJb2'}}, {'type': 'text', 'text': {'value': 'The bar chart for the profit by country has been created and saved as "finance-comparision.png". You can download it using the link below:\n\n[Download finance-comparision.png](sandbox:/mnt/data/finance-comparision.png)', 'annotations': [{'type': 'file_path', 'text': 'sandbox:/mnt/data/finance-comparision.png', 'start_index': 175, 'end_index': 216, 'file